## 导入包

## transformer结构

In [1]:
from IPython.core.display import HTML

# 假设 image_file 是你的图像文件路径
image_file = '../images/The Transformer - model architecture.png'

# 显示图像并居中，同时设置图像的宽度和高度
HTML('<img src="{}" style="display: block; margin-left: auto; margin-right: auto; width: 50%; height: auto;"/>'.format(image_file))

## 多头注意力机制
通过并行计算多个注意力头来捕捉输入序列中不同位置的信息
1. 捕捉长距离依赖

功能：多头注意力机制能够捕捉序列中任意两个位置之间的依赖关系，而不仅仅是局部的或相邻的位置。这使得模型能够更好地理解长距离的上下文信息。

示例：在机器翻译任务中，一个单词可能依赖于句子中的另一个较远位置的单词。多头注意力机制可以同时考虑这些长距离的依赖关系，从而生成更准确的翻译。

2. 并行计算

功能：与循环神经网络（RNN）不同，多头注意力机制可以并行处理输入序列中的所有位置，这大大提高了计算效率，特别是在现代 GPU 和 TPU 上。

示例：在处理长序列时，RNN 需要逐个处理每个位置，而多头注意力机制可以同时处理所有位置，显著减少了训练时间。

3. 捕捉多种特征

功能：通过多个不同的注意力头，模型可以同时学习不同子空间中的特征。每个头可以关注不同的信息，从而捕捉到更丰富的上下文信息。

示例：一个头可能关注语法结构，另一个头可能关注语义信息，还有一个头可能关注上下文中的实体关系。这种多样性和互补性使得模型能够更全面地理解输入序列。

4. 动态权重分配

功能：多头注意力机制通过计算注意力权重，动态地分配每个位置的重要性。这使得模型能够根据上下文动态调整对不同位置的关注程度。

示例：在处理一个句子时，模型可以动态地决定哪些单词对当前单词的生成更重要，从而生成更自然和准确的输出。

5. 支持多种任务

功能：多头注意力机制不仅适用于机器翻译任务，还可以广泛应用于其他自然语言处理任务，如文本生成、问答系统、文本分类等。

示例：在问答系统中，多头注意力机制可以帮助模型更好地理解问题和上下文之间的关系，从而生成更准确的答案。

6. 支持掩码操作

功能：多头注意力机制支持掩码操作，可以屏蔽某些位置的注意力权重，防止模型看到未来的信息或不必要的信息。

示例：在解码器中，掩码操作可以防止模型看到未来的位置，从而确保生成的序列是自回归的。在某些任务中，掩码操作还可以用于处理不完整的输入或特殊的数据结构。

## qkv_attention_value

In [2]:
import torch

def qkv_attention_value(q, k, v, mask=False):
    """
    计算attention value。
    
参数解释：英翻法
1. 批次大小（N）
定义：批次大小 N 表示每次处理的样本数量。
示例：假设我们有 2 个样本，那么 N=2。
2. 查询序列长度（T1）
定义：查询序列长度 T1 表示解码器（Decoder）当前已生成的目标语言序列的长度。
示例：假设解码器已经生成了 3 个法文单词，那么 T1=3。
3. 键/值序列长度（T2）
定义：键/值序列长度 T2 表示编码器（Encoder）输出的源语言序列的长度。
示例：假设源语言（英文）序列有 4 个单词，那么 T2=4。
4. 特征维度（E）
定义：特征维度 E 表示每个 token 的嵌入维度。
示例：假设每个 token 的嵌入维度为 512，那么 E=512。
5. 头的数量（h）
定义：头的数量 h 表示多头注意力机制中的头数。
示例：假设我们使用 8 个头，那么 h=8。
将 512 维的输入分成 8 份，每份 64 维，分别进行计算，从而实现并行学习
输出矩阵中的每个向量包含了该 token 与其他 token 之间的相关性信息

    :param q: 查询向量，形状为[N, T1, E]或[N, h, T1, E]，其中N是批次大小，T1是查询序列长度，E是特征维度，h是头数。
    :param k: 键向量，形状为[N, T2, E]或[N, h, T2, E]，其中T2是键序列长度。
    :param v: 值向量，形状与键向量相同。
    :param mask: 可选的掩码，布尔值或张量，用于屏蔽某些位置的注意力权重。
    :return: 经过注意力加权的值向量，形状为[N, T1, E]或[N, h, T1, E]。
    """
    # 2. 计算q和k之间的相关性->F函数
    k = torch.transpose(k, dim0=-2, dim1=-1)  # [??, T2, E] --> [??, E, T2]，交换最后两个维度以便于矩阵乘法
    # matmul([??,T1,E], [??,E,T2])
    scores = torch.matmul(q, k)  # [??,T1,T2]，计算查询和键之间的点积

    if isinstance(mask, bool):
        if mask:
            # 如果mask为True，则创建一个上三角掩码矩阵
            _shape = scores.shape
            mask = torch.ones((_shape[-2], _shape[-1]))
            mask = torch.triu(mask, diagonal=1) * -10000
            mask = mask[None][None]  # 增加批次和头维度
        else:
            mask = None
    # print (f"mask: {mask}")
    if mask is not None:
        # 将掩码应用到分数上，屏蔽的位置会被赋予一个非常大的负数
        scores = scores + mask

    # 3. 转换为权重
    alpha = torch.softmax(scores, dim=-1)  # [??,T1,T2]，对分数应用softmax得到注意力权重

    # 4. 值的合并
    # matmul([??,T1,T2], [??,T2,E])
    v = torch.matmul(alpha, v)  # [??,T1,E]，使用注意力权重加权值向量
    return v

# 测试示例
if __name__ == "__main__":
    # 定义查询、键和值的维度
    N, T1, T2, E = 2, 3, 4, 5  # 批次大小，查询序列长度，键序列长度，特征维度
    # 创建随机数据
    q = torch.rand(N, T1, E)
    k = torch.rand(N, T2, E)
    v = torch.rand(N, T2, E)
    
    # 调用函数
    attention_value = qkv_attention_value(q, k, v)
    
    # 打印结果
    print("Attention Value Shape:", attention_value.shape)

Attention Value Shape: torch.Size([2, 3, 5])


In [3]:

mask = torch.ones((T1, T2))
print (mask)
print (torch.triu(mask, diagonal=1))
mask = torch.triu(mask, diagonal=1) * -10000
print (mask)
mask = mask[None][None]
print (mask)

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])
tensor([[0., 1., 1., 1.],
        [0., 0., 1., 1.],
        [0., 0., 0., 1.]])
tensor([[    -0., -10000., -10000., -10000.],
        [    -0.,     -0., -10000., -10000.],
        [    -0.,     -0.,     -0., -10000.]])
tensor([[[[    -0., -10000., -10000., -10000.],
          [    -0.,     -0., -10000., -10000.],
          [    -0.,     -0.,     -0., -10000.]]]])


## MultiHeadSelfAttention

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MultiHeadSelfAttention(nn.Module):
    def __init__(self, hidden_size, num_header):
        """
        初始化多头自注意力模块。
        
        :param hidden_size: 隐藏层的大小，也就是特征向量的维度E。
        :param num_header: 多头注意力机制中的头数。
        """
        super(MultiHeadSelfAttention, self).__init__()
        assert hidden_size % num_header == 0, f"hidden_size必须能被num_header整除，当前值分别为：{hidden_size}, {num_header}"
        
        self.hidden_size = hidden_size
        self.num_header = num_header
        
        # 定义Q, K, V的线性变换层
        self.wq = nn.Linear(in_features=self.hidden_size, out_features=self.hidden_size)
        self.wk = nn.Linear(in_features=self.hidden_size, out_features=self.hidden_size)
        self.wv = nn.Linear(in_features=self.hidden_size, out_features=self.hidden_size)
        
        # 定义输出的线性变换层和激活函数
        self.wo = nn.Sequential(
            nn.Linear(in_features=self.hidden_size, out_features=self.hidden_size),
            nn.ReLU()
        )

    def split(self, vs):
        """
        将输入的特征向量分割成多个头。
        
        :param vs: 输入的特征向量，形状为[N, T, E]。
        :return: 分割后的特征向量，形状为[N, H, T, E/H]。
        """
        n, t, e = vs.shape
        vs = torch.reshape(vs, shape=(n, t, self.num_header, e // self.num_header))
        vs = vs.permute(0, 2, 1, 3)  # 调整维度顺序以匹配多头注意力的输入要求
        return vs

    def forward(self, x, attention_mask=None, **kwargs):
        """
        前向传播过程。
        
        :param x: 输入特征向量，形状为[N, T, E]。
        :param attention_mask: 可选的注意力掩码，形状为[N, T, T]。
        :return: 输出特征向量，形状为[N, T, E]。
        """
        # 1. 计算Q, K, V
        q = self.wq(x)  # [N, T, E]
        k = self.wk(x)  # [N, T, E]
        v = self.wv(x)  # [N, T, E]
        
        # 2. 分割Q, K, V以适应多头注意力
        q = self.split(q)  # [N, H, T, E/H]
        k = self.split(k)  # [N, H, T, E/H]
        v = self.split(v)  # [N, H, T, E/H]
        
        # 3. 计算多头自注意力的值
        v = qkv_attention_value(q, k, v, attention_mask)
        
        # 4. 合并多头注意力的结果
        v = v.permute(0, 2, 1, 3)  # [N, H, T, E/H] -> [N, T, H, E/H]
        n, t, _, _ = v.shape
        v = torch.reshape(v, shape=(n, t, -1))  # [N, T, H, E/H] -> [N, T, E]
        v = self.wo(v)  # [N, T, E]
        
        return v

# 测试示例
if __name__ == "__main__":
    # 定义隐藏层大小和头数
    hidden_size = 512
    num_header = 8
    
    # 创建多头自注意力模块实例
    attention = MultiHeadSelfAttention(hidden_size, num_header)
    
    # 创建随机输入数据
    N, T, E = 2, 10, hidden_size  # 批次大小，序列长度，特征维度
    x = torch.rand(N, T, E)
    
    # 调用前向传播
    output = attention(x)
    
    # 打印输出形状
    print("Output shape:", output.shape)

Output shape: torch.Size([2, 10, 512])


## MultiHeadEncoderDecoderAttention

In [22]:
import torch
import torch.nn as nn

class MultiHeadEncoderDecoderAttention(nn.Module):
    """
    多头编码器-解码器注意力模块。
    
    该模块实现了Transformer模型中的编码器-解码器注意力机制，允许解码器动态地聚焦于编码器的输出。
    """
    def __init__(self, hidden_size, num_header):
        super(MultiHeadEncoderDecoderAttention, self).__init__()
        assert hidden_size % num_header == 0, f"hidden_size必须能被num_header整除，当前值分别为：{hidden_size}, {num_header}"
        
        self.hidden_size = hidden_size  # 隐藏层大小，即特征向量的维度E
        self.num_header = num_header  # 头的数目

        # 输出层，将多个头的特征组合合并
        self.wo = nn.Sequential(
            nn.Linear(in_features=self.hidden_size, out_features=self.hidden_size),
            nn.ReLU()
        )

    def split(self, vs):
        """
        将输入的特征向量分割成多个头。
        
        :param vs: 输入的特征向量，形状为[N, T, E]。
        :return: 分割后的特征向量，形状为[N, H, T, E/H]。
        """
        n, t, e = vs.shape
        vs = torch.reshape(vs, shape=(n, t, self.num_header, e // self.num_header))
        vs = torch.permute(vs, dims=(0, 2, 1, 3))
        return vs

    def forward(self, q, encoder_k, encoder_v, encoder_attention_mask, **kwargs):
        """
        前向传播过程。
        
        :param q: 查询向量，形状为[N, T1, E]。
        :param encoder_k: 编码器的键向量，形状为[N, T2, E]。
        :param encoder_v: 编码器的值向量，形状为[N, T2, E]。
        :param encoder_attention_mask: 布尔值或张量，编码器的注意力掩码，形状为[N, 1, T1, T2]。
        :return: 输出特征向量，形状为[N, T1, E]。
        """
        q = self.split(q)  # [N, T1, E] --> [N, H, T1, E/H]
        k = self.split(encoder_k)  # [N, T2, E] --> [N, H, T2, E/H]
        v = self.split(encoder_v)  # [N, T2, E] --> [N, H, T2, E/H]

        # 计算注意力值
        v = qkv_attention_value(q, k, v, mask=encoder_attention_mask)

        # 合并多头注意力的结果
        v = torch.permute(v, dims=(0, 2, 1, 3))  # [N, H, T1, E/H] --> [N, T1, H, E/H]
        n, t, _, _ = v.shape
        v = torch.reshape(v, shape=(n, t, -1))  # [N, T1, H, E/H] --> [N, T1, E]
        v = self.wo(v)  # 应用输出层
        return v

# 测试示例
if __name__ == "__main__":
    # 定义隐藏层大小和头数
    hidden_size = 512
    num_header = 8
    
    # 创建多头编码器-解码器注意力模块实例
    attention = MultiHeadEncoderDecoderAttention(hidden_size, num_header)
    
    # 创建随机输入数据
    N, T1, T2 = 2, 10, 15  # 批次大小，解码器序列长度，编码器序列长度
    q = torch.rand(N, T1, hidden_size)  # 查询向量
    encoder_k = torch.rand(N, T2, hidden_size)  # 编码器键向量
    encoder_v = torch.rand(N, T2, hidden_size)  # 编码器值向量
    
    # 调用前向传播
    output = attention(q, encoder_k, encoder_v, encoder_attention_mask=False)
    
    # 打印输出形状
    print("Output shape:", output.shape)

Output shape: torch.Size([2, 10, 512])


## FFN

In [6]:
import torch
import torch.nn as nn

class FFN(nn.Module):
    """
    位置前馈网络（Feed Forward Network），也称为全连接前馈网络。
    
    该网络在Transformer模型中用于对序列中的每个位置进行相同的操作，通常位于自注意力层之后。
    它由两个线性变换和一个ReLU激活函数组成，第一个线性层将输入特征扩展四倍，第二个线性层将特征还原。
    """
    def __init__(self, hidden_size):
        """
        初始化FFN模块。
        
        :param hidden_size: 输入和输出特征的维度。
        """
        super(FFN, self).__init__()

        # 定义前馈网络的结构
        self.ffn = nn.Sequential(
            nn.Linear(hidden_size, 4 * hidden_size),  # 第一个线性层，将特征维度扩展四倍
            nn.ReLU(),  # ReLU激活函数
            nn.Linear(4 * hidden_size, hidden_size)  # 第二个线性层，将特征维度还原
        )

    def forward(self, x, **kwargs):
        """
        前向传播过程。
        
        :param x: 输入特征，形状为[batch_size, seq_len, hidden_size]。
        :return: 输出特征，形状为[batch_size, seq_len, hidden_size]。
        """
        return self.ffn(x)

# 测试示例
if __name__ == "__main__":
    # 定义隐藏层大小
    hidden_size = 512
    
    # 创建FFN模块实例
    ffn = FFN(hidden_size)
    
    # 创建随机输入数据
    batch_size, seq_len = 2, 10  # 批次大小，序列长度
    x = torch.rand(batch_size, seq_len, hidden_size)  # 输入特征
    
    # 调用前向传播
    output = ffn(x)
    
    # 打印输出形状
    print("Output shape:", output.shape)

Output shape: torch.Size([2, 10, 512])


## ResidualsNorm

In [7]:
import torch
import torch.nn as nn

class ResidualsNorm(nn.Module):
    """
    残差连接和层归一化模块。
    
    该模块结合了残差连接和层归一化（Layer Normalization），常用于Transformer模型中。
    残差连接有助于缓解深层网络中的梯度消失问题，而层归一化则有助于稳定训练过程。
    """
    def __init__(self, block, hidden_size):
        """
        初始化ResidualsNorm模块。
        
        :param block: 一个子模块，其输出将与输入相加（残差连接）。
        :param hidden_size: 归一化层的特征维度。
        """
        super(ResidualsNorm, self).__init__()
        self.block = block  # 子模块
        self.norm = nn.LayerNorm(normalized_shape=hidden_size)  # 层归一化
        self.relu = nn.ReLU()  # ReLU激活函数

    def forward(self, x, **kwargs):
        """
        前向传播过程。
        
        :param x: 输入特征。
        :return: 经过残差连接和层归一化后的特征。
        """
        z = self.block(x, **kwargs)  # 通过子模块处理输入
        z = self.relu(x + z)  # 应用残差连接和ReLU激活函数
        z = self.norm(z)  # 应用层归一化
        return z

# 测试示例
if __name__ == "__main__":
    # 定义隐藏层大小
    hidden_size = 512
    
    # 创建一个示例子模块，例如一个简单的线性层
    example_block = nn.Linear(hidden_size, hidden_size)
    
    # 创建ResidualsNorm模块实例
    residuals_norm = ResidualsNorm(example_block, hidden_size)
    
    # 创建随机输入数据
    batch_size, seq_len = 2, 10  # 批次大小，序列长度
    x = torch.rand(batch_size, seq_len, hidden_size)  # 输入特征
    
    # 调用前向传播
    output = residuals_norm(x)
    
    # 打印输出形状
    print("Output shape:", output.shape)

Output shape: torch.Size([2, 10, 512])


## TransformerEncoderLayers

In [19]:
import torch
import torch.nn as nn

class TransformerEncoderLayers(nn.Module):
    """
    Transformer编码器层组合模块。
    
    该模块将多个编码器层（包含多头自注意力和前馈网络）组合在一起，构成完整的Transformer编码器。
    每个编码器层包括一个残差连接和层归一化的多头自注意力模块，以及一个残差连接和层归一化的前馈网络模块。
    """
    def __init__(self, hidden_size, num_header, encoder_layers):
        """
        初始化Transformer编码器层组合模块。
        
        :param hidden_size: 隐藏层大小，即特征向量的维度。
        :param num_header: 多头自注意力机制中的头数。
        :param encoder_layers: 编码器层的数量。
        """
        super(TransformerEncoderLayers, self).__init__()

        layers = []
        for i in range(encoder_layers):
            layer = [
                ResidualsNorm(
                    block=MultiHeadSelfAttention(hidden_size=hidden_size, num_header=num_header),
                    hidden_size=hidden_size
                ),
                ResidualsNorm(
                    block=FFN(hidden_size=hidden_size),
                    hidden_size=hidden_size
                )
            ]
            layers.extend(layer)
        self.layers = nn.ModuleList(layers)

    def forward(self, x, attention_mask):
        """
        前向传播过程。
        
        :param x: 输入特征，形状为[batch_size, seq_len, hidden_size]。
        :param attention_mask: 注意力掩码，形状为[batch_size, seq_len]。
        :return: 经过所有编码器层处理后的特征。
        """
        attention_mask = torch.unsqueeze(attention_mask, dim=1)  # 增加header维度
        print (f"encoder attention_mask.shape: {attention_mask.shape}")
        for layer in self.layers:
            x = layer(x, attention_mask=attention_mask)
        return x

# 示例代码
if __name__ == "__main__":
    # 定义隐藏层大小、头数和编码器层数
    hidden_size = 512
    num_header = 8
    encoder_layers = 6
    
    # 创建Transformer编码器层组合模块实例
    encoder_layers_module = TransformerEncoderLayers(hidden_size, num_header, encoder_layers)
    
    # 创建随机输入数据和注意力掩码
    batch_size, seq_len = 2, 10  # 批次大小，序列长度
    x = torch.rand(batch_size, seq_len, hidden_size)  # 输入特征
    attention_mask = torch.ones(batch_size, seq_len, seq_len)  # 注意力掩码，全1表示无掩码
    
    # 调用前向传播
    output = encoder_layers_module(x, attention_mask)
    
    # 打印输出形状
    print("Output shape:", output.shape)

encoder attention_mask.shape: torch.Size([2, 1, 10, 10])
Output shape: torch.Size([2, 10, 512])


In [9]:
print (encoder_layers_module)

TransformerEncoderLayers(
  (layers): ModuleList(
    (0): ResidualsNorm(
      (block): MultiHeadSelfAttention(
        (wq): Linear(in_features=512, out_features=512, bias=True)
        (wk): Linear(in_features=512, out_features=512, bias=True)
        (wv): Linear(in_features=512, out_features=512, bias=True)
        (wo): Sequential(
          (0): Linear(in_features=512, out_features=512, bias=True)
          (1): ReLU()
        )
      )
      (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (relu): ReLU()
    )
    (1): ResidualsNorm(
      (block): FFN(
        (ffn): Sequential(
          (0): Linear(in_features=512, out_features=2048, bias=True)
          (1): ReLU()
          (2): Linear(in_features=2048, out_features=512, bias=True)
        )
      )
      (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (relu): ReLU()
    )
    (2): ResidualsNorm(
      (block): MultiHeadSelfAttention(
        (wq): Linear(in_features=512, out_features

## TransformerEncoder

In [20]:
import torch
import torch.nn as nn

class TransformerEncoder(nn.Module):
    """
    Transformer编码器模块。
    
    该模块实现了Transformer模型中的编码器部分，包括嵌入层、位置编码和编码器层。
    """
    def __init__(self, vocab_size, hidden_size, num_header, max_seq_length, encoder_layers):
        """
        初始化Transformer编码器模块。
        
        :param vocab_size: 词汇表大小，用于定义嵌入层的词汇表大小。
        :param hidden_size: 隐藏层大小，即特征向量的维度。
        :param num_header: 多头自注意力机制中的头数。
        :param max_seq_length: 序列的最大长度，用于定义位置嵌入的大小。
        :param encoder_layers: 编码器层的数量。
        """
        super(TransformerEncoder, self).__init__()

        self.input_emb = nn.Embedding(num_embeddings=vocab_size, embedding_dim=hidden_size)  # 嵌入层
        self.position_emb = nn.Embedding(num_embeddings=max_seq_length, embedding_dim=hidden_size)  # 位置嵌入层
        self.layers = TransformerEncoderLayers(hidden_size, num_header, encoder_layers)  # 编码器层

    def forward(self, input_token_ids, input_position_ids, input_mask):
        """
        前向传播过程。
        
        :param input_token_ids: 输入的token id，形状为[N, T]，long类型。
        :param input_position_ids: 输入的位置id，形状为[N, T]，long类型。
        :param input_mask: 输入的mask矩阵，形状为[N, T, T]，float类型。
        :return: 编码器的输出特征。
        """
        # 1. 获取token的embedding
        inp_embedding = self.input_emb(input_token_ids)  # [N, T, E]

        # 2. 获取位置embedding
        position_embedding = self.position_emb(input_position_ids)

        # 3. 合并embedding
        emd = inp_embedding + position_embedding

        # 4. 输入到编码器层提取特征
        feat_emd = self.layers(emd, attention_mask=input_mask)

        return feat_emd

# 测试代码
if __name__ == "__main__":
    # 定义参数
    vocab_size = 10000
    hidden_size = 512
    num_header = 8
    max_seq_length = 100
    encoder_layers = 6
    
    # 创建Transformer编码器模块实例
    encoder = TransformerEncoder(vocab_size, hidden_size, num_header, max_seq_length, encoder_layers)
    
    # 创建随机输入数据
    batch_size, seq_len = 2, 10  # 批次大小，序列长度
    input_token_ids = torch.randint(0, vocab_size, (batch_size, seq_len))  # 输入的token id
    input_position_ids = torch.arange(seq_len).repeat(batch_size, 1)  # 输入的位置id
    input_mask = torch.ones(batch_size, seq_len, seq_len)  # 输入的mask矩阵
    
    # 调用前向传播
    output = encoder(input_token_ids, input_position_ids, input_mask)
    
    # 打印输出形状
    print("Output shape:", output.shape)

encoder attention_mask.shape: torch.Size([2, 1, 10, 10])
Output shape: torch.Size([2, 10, 512])


## TransformerDecoderLayers

In [11]:
import torch
import torch.nn as nn

class TransformerDecoderLayers(nn.Module):
    """
    Transformer解码器层组合模块。
    
    该模块将多个解码器层（包含自注意力、编码器-解码器注意力和前馈网络）组合在一起，构成完整的Transformer解码器层。
    """
    def __init__(self, hidden_size, num_header, decoder_layers):
        """
        初始化Transformer解码器层组合模块。
        
        :param hidden_size: 隐藏层大小，即特征向量的维度。
        :param num_header: 多头自注意力机制中的头数。
        :param decoder_layers: 解码器层的数量。
        """
        super(TransformerDecoderLayers, self).__init__()

        # 定义键和值的线性变换层
        self.wk = nn.Linear(hidden_size, hidden_size)
        self.wv = nn.Linear(hidden_size, hidden_size)

        layers = []
        for i in range(decoder_layers):
            layer = [
                ResidualsNorm(
                    block=MultiHeadSelfAttention(hidden_size=hidden_size, num_header=num_header),
                    hidden_size=hidden_size
                ),
                ResidualsNorm(
                    block=MultiHeadEncoderDecoderAttention(hidden_size=hidden_size, num_header=num_header),
                    hidden_size=hidden_size
                ),
                ResidualsNorm(
                    block=FFN(hidden_size=hidden_size),
                    hidden_size=hidden_size
                )
            ]
            layers.extend(layer)
        self.layers = nn.ModuleList(layers)

    def forward(self, x, encoder_outputs=None, encoder_attention_mask=None, attention_mask=None):
        """
        前向传播过程。
        
        :param x: 输入特征，形状为[N, T2, E]。
        :param encoder_outputs: 编码器的输出，形状为[N, T1, E]。
        :param encoder_attention_mask: 编码器的注意力掩码，形状为[N, 1, T1]。
        :param attention_mask: 解码器的自注意力掩码，形状为[N, T2, T2]。
        :return: 解码器层的输出特征。
        """
        # 增加维度以匹配多头注意力的输入要求
        if not isinstance(attention_mask, bool):
            attention_mask = torch.unsqueeze(attention_mask, dim=1)  # [N, T2, T2] -> [N, 1, T2, T2]
            print (f"decoder attention_mask.shape: {attention_mask.shape}")
        if not isinstance(encoder_attention_mask, bool):
            encoder_attention_mask = torch.unsqueeze(encoder_attention_mask, dim=1)  # [N, 1, T1] -> [N, 1, 1, T1]
            print (f"decoder encoder_attention_mask.shape: {encoder_attention_mask.shape}")
        
        # 通过线性层计算编码器输出的键和值
        k = self.wk(encoder_outputs)  # [N, T1, E]
        v = self.wv(encoder_outputs)  # [N, T1, E]
        
        # 遍历每个解码器层
        for layer in self.layers:
            x = layer(
                x,
                encoder_k=k, encoder_v=v, encoder_attention_mask=encoder_attention_mask,
                attention_mask=attention_mask
            )
        return x

# 示例代码
if __name__ == "__main__":
    # 定义参数
    hidden_size = 512
    num_header = 8
    decoder_layers = 6
    vocab_size = 10000
    max_seq_length = 100
    
    # 创建Transformer解码器层组合模块实例
    decoder_layers_module = TransformerDecoderLayers(hidden_size, num_header, decoder_layers)
    
    # 创建随机输入数据
    batch_size, seq_len = 2, 10  # 批次大小，序列长度
    x = torch.rand(batch_size, seq_len, hidden_size)  # 输入特征
    encoder_outputs = torch.rand(batch_size, seq_len, hidden_size)  # 编码器的输出
    # encoder_attention_mask = torch.ones(batch_size, 1, seq_len)  # 编码器的注意力掩码
    # attention_mask = torch.ones(batch_size, seq_len, seq_len)  # 解码器的自注意力掩码
    encoder_attention_mask = True
    attention_mask = True
    
    # 调用前向传播
    output = decoder_layers_module(x, encoder_outputs, encoder_attention_mask, attention_mask)
    
    # 打印输出形状
    print("Output shape:", output.shape)

Output shape: torch.Size([2, 10, 512])


In [12]:
print (decoder_layers_module)

TransformerDecoderLayers(
  (wk): Linear(in_features=512, out_features=512, bias=True)
  (wv): Linear(in_features=512, out_features=512, bias=True)
  (layers): ModuleList(
    (0): ResidualsNorm(
      (block): MultiHeadSelfAttention(
        (wq): Linear(in_features=512, out_features=512, bias=True)
        (wk): Linear(in_features=512, out_features=512, bias=True)
        (wv): Linear(in_features=512, out_features=512, bias=True)
        (wo): Sequential(
          (0): Linear(in_features=512, out_features=512, bias=True)
          (1): ReLU()
        )
      )
      (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (relu): ReLU()
    )
    (1): ResidualsNorm(
      (block): MultiHeadEncoderDecoderAttention(
        (wo): Sequential(
          (0): Linear(in_features=512, out_features=512, bias=True)
          (1): ReLU()
        )
      )
      (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (relu): ReLU()
    )
    (2): ResidualsNorm(
      (bl

## TransformerDecoder

In [13]:
import torch
import torch.nn as nn

class TransformerDecoder(nn.Module):
    """
    Transformer解码器模块。
    
    该模块实现了Transformer模型中的解码器部分，包括嵌入层、位置编码和解码器层。
    """
    def __init__(self, vocab_size, hidden_size, num_header, max_seq_length, decoder_layers):
        """
        初始化Transformer解码器模块。
        
        :param vocab_size: 词汇表大小，用于定义嵌入层的词汇表大小。
        :param hidden_size: 隐藏层大小，即特征向量的维度。
        :param num_header: 多头自注意力机制中的头数。
        :param max_seq_length: 序列的最大长度，用于定义位置嵌入的大小。
        :param decoder_layers: 解码器层的数量。
        """
        super(TransformerDecoder, self).__init__()

        self.input_emb = nn.Embedding(num_embeddings=vocab_size, embedding_dim=hidden_size)  # 嵌入层
        self.position_emb = nn.Embedding(num_embeddings=max_seq_length, embedding_dim=hidden_size)  # 位置嵌入层
        self.layers = TransformerDecoderLayers(hidden_size, num_header, decoder_layers)  # 解码器层

    def forward(self, input_token_ids, input_position_ids, input_mask, encoder_outputs, encoder_attention_mask):
        """
        前向传播过程。
        
        :param input_token_ids: 输入的token id，形状为[N, T]，long类型。
        :param input_position_ids: 输入的位置id，形状为[N, T]，long类型。
        :param input_mask: 输入的mask矩阵，形状为[N, T, T]，float类型。
        :param encoder_outputs: 编码器的输出状态信息，形状为[N, T1, E]。
        :param encoder_attention_mask: 编码器的输入mask信息，形状为[N, T1, T1]。
        :return: 解码器的输出特征。
        """
        if self.training:
            # 1. 获取token的embedding
            inp_embedding = self.input_emb(input_token_ids)  # [N, T, E]

            # 2. 获取位置embedding
            position_embedding = self.position_emb(input_position_ids)

            # 3. 合并embedding
            emd = inp_embedding + position_embedding

            # 4. 输入到解码器层提取特征
            feat_emd = self.layers(
                emd, encoder_outputs=encoder_outputs,
                encoder_attention_mask=encoder_attention_mask, attention_mask=input_mask
            )

            return feat_emd
        else:
            raise ValueError("当前模拟代码不实现推理过程，仅实现training过程")

# 示例代码
if __name__ == "__main__":
    # 定义参数
    vocab_size = 10000
    hidden_size = 512
    num_header = 8
    max_seq_length = 100
    decoder_layers = 6
    
    # 创建Transformer解码器模块实例
    decoder = TransformerDecoder(vocab_size, hidden_size, num_header, max_seq_length, decoder_layers)
    
    # 创建随机输入数据
    batch_size, seq_len = 2, 10  # 批次大小，序列长度
    input_token_ids = torch.randint(0, vocab_size, (batch_size, seq_len))  # 输入的token id
    input_position_ids = torch.arange(seq_len).repeat(batch_size, 1)  # 输入的位置id
    encoder_outputs = torch.rand(batch_size, seq_len, hidden_size)  # 编码器的输出状态信息
    input_mask = torch.ones(batch_size, seq_len, seq_len)  # 输入的mask矩阵
    encoder_attention_mask = torch.ones(batch_size, seq_len, seq_len)  # 编码器的输入mask信息
    
    # 调用前向传播
    output = decoder(input_token_ids, input_position_ids, input_mask, encoder_outputs, encoder_attention_mask)
    
    # 打印输出形状
    print("Output shape:", output.shape)

decoder attention_mask.shape: torch.Size([2, 1, 10, 10])
decoder encoder_attention_mask.shape: torch.Size([2, 1, 10, 10])
Output shape: torch.Size([2, 10, 512])


In [ ]:
y=1

## TransformerEncoder & TransformerDecoder

In [14]:
encoder = TransformerEncoder(vocab_size=1000, hidden_size=512, num_header=8, max_seq_length=1024, encoder_layers=6)
decoder = TransformerDecoder(vocab_size=1000, hidden_size=512, num_header=8, max_seq_length=1024, decoder_layers=6)

input_token_ids = torch.tensor([
    [100, 102, 108, 253, 125],  # 第一个样本实际长度为5
    [254, 125, 106, 0, 0]  # 第二个样本实际长度为3
])
input_position_ids = torch.tensor([
    [0, 1, 2, 3, 4],
    [0, 1, 2, 3, 4]
])
input_mask = torch.tensor([
    [
        [0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0],
    ],
    [
        [0.0, 0.0, 0.0, -10000.0, -10000.0],
        [0.0, 0.0, 0.0, -10000.0, -10000.0],
        [0.0, 0.0, 0.0, -10000.0, -10000.0],
        [-10000.0, -10000.0, -10000.0, 0.0, -10000.0],
        [-10000.0, -10000.0, -10000.0, -10000.0, 0.0],
    ],
])
encoder_attention_mask = torch.tensor([
    [
        [0.0, 0.0, 0.0, 0.0, 0.0]  # 表示第一个样本的解码器中第一个时刻和编码器的各个时刻之间的mask值
    ],
    [
        [0.0, 0.0, 0.0, -10000.0, -10000.0]  # 是因为编码器的输入中，最后两个位置是填充
    ],
])

input_decoder_token_ids = torch.tensor([
    [251, 235, 124, 321, 25, 68],
    [351, 235, 126, 253, 0, 0]
])
input_decoder_position_ids = torch.tensor([
    [0, 1, 2, 3, 4, 5],
    [0, 1, 2, 3, 4, 5]
])
input_decoder_mask = torch.tensor([
    [
        [0.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0],
        [0.0, 0.0, -10000.0, -10000.0, -10000.0, -10000.0],
        [0.0, 0.0, 0.0, -10000.0, -10000.0, -10000.0],
        [0.0, 0.0, 0.0, 0.0, -10000.0, -10000.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, -10000.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    ],
    [
        [0.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0],
        [0.0, 0.0, -10000.0, -10000.0, -10000.0, -10000.0],
        [0.0, 0.0, 0.0, -10000.0, -10000.0, -10000.0],
        [0.0, 0.0, 0.0, 0.0, -10000.0, -10000.0],
        [-10000.0, -10000.0, -10000.0, -10000.0, 0.0, -10000.0],
        [-10000.0, -10000.0, -10000.0, -10000.0, -10000.0, 0.0]
    ],
])

encoder_outputs = encoder(input_token_ids, input_position_ids, input_mask)
print(encoder_outputs.shape)

decoder_outputs = decoder(
    input_token_ids=input_decoder_token_ids,
    input_position_ids=input_decoder_position_ids,
    input_mask=input_decoder_mask,
    encoder_outputs=encoder_outputs,
    encoder_attention_mask=encoder_attention_mask
)
print(decoder_outputs.shape)

encoder attention_mask.shape: torch.Size([2, 1, 5, 5])
torch.Size([2, 5, 512])
decoder attention_mask.shape: torch.Size([2, 1, 6, 6])
decoder encoder_attention_mask.shape: torch.Size([2, 1, 1, 5])
torch.Size([2, 6, 512])


## Transformer

In [15]:
import torch
import torch.nn as nn

# 假设以下类已经定义好：
# TransformerEncoder, TransformerDecoder, TransformerEncoderLayers, TransformerDecoderLayers
# MultiHeadSelfAttention, MultiHeadEncoderDecoderAttention, FFN, ResidualsNorm

class Transformer(nn.Module):
    """
    完整的Transformer模型，包含编码器和解码器。
    
    该模型实现了序列到序列的转换任务，例如机器翻译。
    """
    def __init__(self, encoder_vocab_size, decoder_vocab_size, hidden_size, num_heads, max_seq_length, encoder_layers, decoder_layers):
        """
        初始化Transformer模型。
        
        :param encoder_vocab_size: 编码器词汇表大小。
        :param decoder_vocab_size: 解码器词汇表大小。
        :param hidden_size: 隐藏层大小。
        :param num_heads: 多头自注意力机制中的头数。
        :param max_seq_length: 序列的最大长度。
        :param encoder_layers: 编码器层的数量。
        :param decoder_layers: 解码器层的数量。
        """
        super(Transformer, self).__init__()

        # 初始化编码器
        self.encoder = TransformerEncoder(
            vocab_size=encoder_vocab_size, hidden_size=hidden_size,
            num_header=num_heads, max_seq_length=max_seq_length,
            encoder_layers=encoder_layers
        )
        
        # 初始化解码器
        self.decoder = TransformerDecoder(
            vocab_size=decoder_vocab_size, hidden_size=hidden_size,
            num_header=num_heads, max_seq_length=max_seq_length,
            decoder_layers=decoder_layers
        )

    def forward(self, encoder_input_ids, encoder_input_position_ids, encoder_input_mask,
                decoder_input_ids, decoder_input_position_ids, decoder_input_mask, encoder_attention_mask):
        """
        前向传播过程。
        
        :param encoder_input_ids: 编码器输入的token id，形状为[N, T]。
        :param encoder_input_position_ids: 编码器输入的位置id，形状为[N, T]。
        :param encoder_input_mask: 编码器输入的mask矩阵，形状为[N, T, T]。
        :param decoder_input_ids: 解码器输入的token id，形状为[N, T]。
        :param decoder_input_position_ids: 解码器输入的位置id，形状为[N, T]。
        :param decoder_input_mask: 解码器输入的mask矩阵，形状为[N, T, T]。
        :param encoder_attention_mask: 编码器注意力掩码，形状为[N, 1, T]。
        :return: 解码器的输出分数。
        """
        # 编码器前向传播
        encoder_outputs = self.encoder(encoder_input_ids, encoder_input_position_ids, encoder_input_mask)
        
        # 解码器前向传播
        decoder_outputs = self.decoder(
            input_token_ids=decoder_input_ids,
            input_position_ids=decoder_input_position_ids,
            input_mask=decoder_input_mask,
            encoder_outputs=encoder_outputs,
            encoder_attention_mask=encoder_attention_mask
        )
        
        return decoder_outputs

In [16]:


# 实例化Transformer模型
transformer = Transformer(
    encoder_vocab_size=1000, decoder_vocab_size=1000, hidden_size=512, num_heads=8,
    max_seq_length=1024, encoder_layers=6, decoder_layers=6
)

# 创建输入数据
input_token_ids = torch.tensor([
    [100, 102, 108, 253, 125],  # 第一个样本实际长度为5
    [254, 125, 106, 0, 0]  # 第二个样本实际长度为3
])
input_position_ids = torch.tensor([
    [0, 1, 2, 3, 4],
    [0, 1, 2, 3, 4]
])
input_mask = torch.tensor([
    [
        [0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0],
    ],
    [
        [0.0, 0.0, 0.0, -10000.0, -10000.0],
        [0.0, 0.0, 0.0, -10000.0, -10000.0],
        [0.0, 0.0, 0.0, -10000.0, -10000.0],
        [-10000.0, -10000.0, -10000.0, 0.0, -10000.0],
        [-10000.0, -10000.0, -10000.0, -10000.0, 0.0],
    ],
])
encoder_attention_mask = torch.tensor([
    [
        [0.0, 0.0, 0.0, 0.0, 0.0]  # 表示第一个样本的解码器中第一个时刻和编码器的各个时刻之间的mask值
    ],
    [
        [0.0, 0.0, 0.0, -10000.0, -10000.0]  # 是因为编码器的输入中，最后两个位置是填充
    ],
])

input_decoder_token_ids = torch.tensor([
    [251, 235, 124, 321, 25, 68],
    [351, 235, 126, 253, 0, 0]
])
input_decoder_position_ids = torch.tensor([
    [0, 1, 2, 3, 4, 5],
    [0, 1, 2, 3, 4, 5]
])
input_decoder_mask = torch.tensor([
    [
        [0.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0],
        [0.0, 0.0, -10000.0, -10000.0, -10000.0, -10000.0],
        [0.0, 0.0, 0.0, -10000.0, -10000.0, -10000.0],
        [0.0, 0.0, 0.0, 0.0, -10000.0, -10000.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, -10000.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    ],
    [
        [0.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0],
        [0.0, 0.0, -10000.0, -10000.0, -10000.0, -10000.0],
        [0.0, 0.0, 0.0, -10000.0, -10000.0, -10000.0],
        [0.0, 0.0, 0.0, 0.0, -10000.0, -10000.0],
        [-10000.0, -10000.0, -10000.0, -10000.0, 0.0, -10000.0],
        [-10000.0, -10000.0, -10000.0, -10000.0, -10000.0, 0.0]
    ],
])

# 执行前向传播
decoder_outputs = transformer(
    encoder_input_ids=input_token_ids, 
    encoder_input_position_ids=input_position_ids, 
    encoder_input_mask=input_mask,
    decoder_input_ids=input_decoder_token_ids,
    decoder_input_position_ids=input_decoder_position_ids,
    decoder_input_mask=input_decoder_mask, 
    encoder_attention_mask=encoder_attention_mask
)
print(decoder_outputs.shape)

encoder attention_mask.shape: torch.Size([2, 1, 5, 5])
decoder attention_mask.shape: torch.Size([2, 1, 6, 6])
decoder encoder_attention_mask.shape: torch.Size([2, 1, 1, 5])
torch.Size([2, 6, 512])


## pytorch transformer

In [1]:
import torch
import torch.nn as nn

# 定义模型的参数
d_model = 512  # 嵌入的维度
nhead = 8  # 多头注意力机制中的头数
num_encoder_layers = 6  # 编码器层数
num_decoder_layers = 6  # 解码器层数
dim_feedforward = 2048  # 前馈网络的维度
dropout = 0.1  # Dropout 比率

# 创建 Transformer 模型
transformer_model = nn.Transformer(
    d_model=d_model,
    nhead=nhead,
    num_encoder_layers=num_encoder_layers,
    num_decoder_layers=num_decoder_layers,
    dim_feedforward=dim_feedforward,
    dropout=dropout
)

# 创建一些随机数据来模拟输入
src = torch.rand(10, 32, d_model)  # 假设的源序列长度为10，批次大小为32
tgt = torch.rand(20, 32, d_model)  # 假设的目标序列长度为20，批次大小为32

# 创建位置编码
src_key_padding_mask = torch.zeros(32, 10)  # 源序列的键填充掩码
tgt_key_padding_mask = torch.zeros(32, 20)  # 目标序列的键填充掩码
memory_key_padding_mask = torch.zeros(32, 10)  # 记忆序列的键填充掩码

# 前向传播
output = transformer_model(src, tgt, src_key_padding_mask=src_key_padding_mask, tgt_key_padding_mask=tgt_key_padding_mask, memory_key_padding_mask=memory_key_padding_mask)

print(output.shape)  # 输出的形状将取决于目标序列和批次大小

/opt/anaconda3/envs/ait/lib/python3.10/site-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


torch.Size([20, 32, 512])


## 补充
- https://github.com/gordicaleksa/pytorch-original-transformer